### Installing Dependencies

In [ ]:
%pip install lime

In [ ]:
import os
import numpy as np
from PIL import Image
from lime import lime_image
from skimage.segmentation import mark_boundaries

import torch
import torch.nn as nn
import matplotlib.pyplot as plt
from torchvision.transforms import Normalize
from torchvision.transforms import ToTensor
from torchvision.transforms import Compose
from torchvision.transforms import Resize


from google.colab import drive
drive.mount('/content/drive')

In [ ]:
means = [0.43766510486602783, 0.49804747104644775, 0.3756938874721527]
stds = [0.16779577732086182, 0.1552586406469345, 0.1632111817598343]

def get_input_transform():
    normalize = Normalize(mean=means, std=stds)
    transf = Compose([
        Resize((64, 64)),
        ToTensor(),
        normalize
    ])
    return transf

def get_input_tensors(img):
    transf = get_input_transform()
    return transf(img).unsqueeze(0)

class LeNet(nn.Module):
    def __init__(self):
        super(LeNet, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.relu1 = nn.ReLU()
        self.pool1 = nn.MaxPool2d(2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.relu2 = nn.ReLU()
        self.pool2 = nn.MaxPool2d(2)
        self.fc1 = nn.Linear(2704, 120)
        self.relu3 = nn.ReLU()
        self.fc2 = nn.Linear(120, 84)
        self.relu4 = nn.ReLU()
        self.fc3 = nn.Linear(84, 4)
        self.relu5 = nn.ReLU()

    def forward(self, x):
        y = self.conv1(x)
        y = self.relu1(y)
        y = self.pool1(y)
        y = self.conv2(y)
        y = self.relu2(y)
        y = self.pool2(y)
        y = y.view(y.shape[0], -1)
        y = self.fc1(y)
        y = self.relu3(y)
        y = self.fc2(y)
        y = self.relu4(y)
        y = self.fc3(y)
        y = self.relu5(y)
        return y

model = LeNet()
checkpoint = torch.load('/content/drive/MyDrive/LeNet_Updated.pth', map_location=torch.device('cpu'))
model.load_state_dict(checkpoint["state_dict"])
model.eval()

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

## Using LIME to Explain the Model

In [ ]:
def get_image(path):
    with open(os.path.abspath(path), 'rb') as f:
        with Image.open(f) as img:
            return img.convert('RGB')
# img = get_image('/content/drive/MyDrive/data/Healthy/Corn_Health (183).jpg') # healthy
img = get_image('/content/drive/MyDrive/data/Gray_Leaf_Spot/Corn_Gray_Spot (135).JPG') #gray spot
plt.imshow(img)

In [ ]:
def get_preprocess_transform():
    normalize = Normalize(mean=means, std=stds)
    transf = Compose([
        Resize((64, 64)),
        ToTensor(),
        normalize
    ])
    return transf

preprocess_transform = get_preprocess_transform()

def predict(input):
    input_tensor = preprocess_transform(input)
    input_batch = input_tensor.unsqueeze(0)

   # input_batch = input_batch.to('cuda')

    with torch.no_grad():
        output = model(input_batch)

    return output[0]

test_pred = predict(img)

categories = ["Blight", "Common Rust", "Gray Leaf Spot", "Healthy"]

print(categories[test_pred.squeeze().argmax()])

In [ ]:
# Function to predict and return probabilities
def batch_predict(images):
    model.eval()
    batch = torch.stack([preprocess_transform(Image.fromarray(image.astype('uint8')).convert('RGB')) for image in images], dim=0)
    batch = batch.to(device)
    logits = model(batch)
    probs = torch.nn.functional.softmax(logits, dim=1)
    return probs.detach().cpu().numpy()

# Use LIME to explain the predictions
explainer = lime_image.LimeImageExplainer()
explanation = explainer.explain_instance(np.array(img), batch_predict, top_labels=1, hide_color=0, num_samples=1000)
temp, mask = explanation.get_image_and_mask(explanation.top_labels[0], positive_only=True, num_features=5, hide_rest=False)

plt.imshow(mark_boundaries(temp / 255.0, mask))
plt.show()

highlight = np.zeros_like(temp)
highlight[mask == 1] = [144, 238, 144]  # Light green color in RGB

# Blend the highlight with the original image (25% opacity)
alpha = 0.25
highlighted_image = np.where(mask[..., np.newaxis], alpha * highlight + (1 - alpha) * temp, temp)

plt.imshow(mark_boundaries(highlighted_image / 255.0, mask))
plt.show()